In [1]:
# Import standard libraries
import os
from contextlib import redirect_stdout

import sys
# append coeqwal packages to path
sys.path.append('./coeqwalpackage')

import numpy as np
import pandas as pd
import datetime as dt

In [2]:
# Import custom libraries
# Note: on my computer the next import doesn't work the first time I call it, why? If I re-run the cell, then it is ok. MUST DEBUG
from coeqwalpackage.metrics import *
import cqwlutils as cu
import plotting as pu

## Define Control File

In [3]:
CtrlFile = 'CalSim3DataExtractionInitFile_v4.xlsx'
CtrlTab = 'Init'

## Read From Control File

In [4]:
ScenarioListFile, ScenarioListTab, ScenarioListPath, DVDssNamesOutPath, SVDssNamesOutPath, ScenarioIndicesOutPath, DssDirsOutPath, VarListPath, VarListFile, VarListTab, VarOutPath, DataOutPath, ConvertDataOutPath, ExtractionSubPath, DemandDeliverySubPath, ModelSubPath, GroupDataDirPath, ScenarioDir, DVDssMin, DVDssMax, SVDssMin, SVDssMax, NameMin, NameMax, DirMin, DirMax, IndexMin, IndexMax, StartMin, StartMax, EndMin, EndMax, VarMin, VarMax, DemandFilePath, DemandFileName, DemandFileTab, DemMin, DemMax, InflowOutSubPath, InflowFilePath, InflowFileName, InflowFileTab, InflowMin, InflowMax = cu.read_init_file(CtrlFile, CtrlTab)

## Read in Data

In [5]:
df, dss_names = read_in_df(ConvertDataOutPath,DVDssNamesOutPath)
df = add_water_year_column(df)

C:\Dino\COEQWAL\COEQWAL_GIT\coeqwal\notebooks\coeqwalpackage\metrics.py:175: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  return df_copy.drop(["Date", "Year", "Month"], axis=1)


## Add constant columns to df

In [6]:
# need a column for S_MLRTNLEVEL5DV with value 524 for each scenario

## Tier Assignments

##### Helper Function

In [7]:
def classify_tier(prob):
    if np.isnan(prob):
        return np.nan
    elif prob < 0.10:
        return "1"
    elif prob < 0.40:
        return "2"
    else:
        return "3"

In [8]:
# ALSO WHICH LUIS TO USE? S_SLUIS_CVP_ or S_SLUIS_SWP_
# TAF units or AF units?
# variables_storage = ["S_SHSTA_", "S_OROVL_", "S_FOLSM_", "S_TRNTY_", "S_SLUIS_CVP_", "S_SLUIS_SWP_", "S_MELON_", "S_MLRTN_"]
# variables_floodpool = ["S_SHSTALEVEL5DV", "S_OROVLLEVEL5DV", "S_FOLSMLEVEL5DV", "S_TRNTYLEVEL5DV", "S_SLUIS_CVPLEVEL5DV", "S_SLUIS_SWPLEVEL5DV", "S_MELON_LEVEL5DV"]
variables_storage = ["S_SHSTA_", "S_OROVL_", "S_FOLSM_", "S_TRNTY_", "S_SLUIS_CVP_", "S_SLUIS_SWP_", "S_MLRTN_"]
variables_floodpool = ["S_SHSTALEVEL5DV", "S_OROVLLEVEL5DV", "S_FOLSMLEVEL5DV", "S_TRNTYLEVEL5DV", "S_SLUIS_CVPLEVEL5DV", "S_SLUIS_SWPLEVEL5DV", "S_MLRTN_LEVEL5DV"]

In [9]:
# hitting floodpool = variables_storage >= variables_floodpool
#FOLSOM is Nan for scenario 6 - 10 because it doesn't have level5D for sceanrio 6-10 "S_FOLSMLEVEL5DV"

prob_result_list = []

for scenario in dss_names:
    scenario_short = scenario[:5]
    row_data = {"Scenario": scenario_short}

    for var_s, var_lvl5D in zip(variables_storage, variables_floodpool):
        var1_name = f"{var_s}{scenario_short}"
        var2_name = f"{var_lvl5D}_{scenario_short}"

        p_gte = probability_var1_gte_var2_for_scenario(df, var1_name, var2_name, units="TAF", tolerance=1e-6)
        if(var_s == "S_MLRTN_"):
            p_gte = probability_var1_gte_const_for_scenario(df, var1_name, 524, units="TAF")
        
        gte_col = f"Prob_{var_s}GTE_{var_lvl5D}"
        tier_col = f"{var_s}Tier"

        row_data[gte_col] = p_gte
        row_data[tier_col] = classify_tier(p_gte)

    prob_result_list.append(row_data)


df_flow_probs = pd.DataFrame(prob_result_list)
df_flow_probs.set_index("Scenario", inplace=True)

display(df_flow_probs)

,Prob_S_SHSTA_GTE_S_SHSTALEVEL5DV,S_SHSTA_Tier,Prob_S_OROVL_GTE_S_OROVLLEVEL5DV,S_OROVL_Tier,Prob_S_FOLSM_GTE_S_FOLSMLEVEL5DV,S_FOLSM_Tier,Prob_S_TRNTY_GTE_S_TRNTYLEVEL5DV,S_TRNTY_Tier,Prob_S_SLUIS_CVP_GTE_S_SLUIS_CVPLEVEL5DV,S_SLUIS_CVP_Tier,Prob_S_SLUIS_SWP_GTE_S_SLUIS_SWPLEVEL5DV,S_SLUIS_SWP_Tier,Prob_S_MLRTN_GTE_S_MLRTN_LEVEL5DV,S_MLRTN_Tier
Scenario,,,,,,,,,,,,,,
s0001,0.333333,2,0.215000,2,0.431667,3,0.000000,1,0.078333,1,0.065833,1,0.023333,1
s0002,0.315000,2,0.210000,2,0.423333,3,0.000833,1,0.076667,1,0.073333,1,0.024167,1
s0003,0.222500,2,0.165833,2,0.339167,2,0.000000,1,0.055000,1,0.060000,1,0.027500,1
s0004,0.192500,2,0.144167,2,0.305000,2,0.000000,1,0.046667,1,0.051667,1,0.028333,1
s0005,0.175833,2,0.133333,2,0.280000,2,0.000000,1,0.041667,1,0.042500,1,0.027500,1
s0006,0.322695,2,0.216312,2,0.406028,3,0.000000,1,0.090426,1,0.061170,1,0.045213,1
s0007,0.320922,2,0.214539,2,0.402482,3,0.000000,1,0.093085,1,0.170213,2,0.045213,1
s0008,0.320035,2,0.212766,2,0.402482,3,0.000000,1,0.093972,1,0.191489,2,0.045213,1
s0009,0.320035,2,0.214539,2,0.401596,3,0.000000,1,0.096631,1,0.201241,2,0.045213,1


In [11]:
output_dir = os.path.join(ScenarioDir, "Performance_Metrics", "Tiered_Outcome_Measures", "Reservoir_FloodRisk")
output_path = os.path.join(output_dir, "tier_assignment_output.csv")

# check if output directory exists
if not os.path.exists(output_dir):
    # print warning
    print("Warning: directory " + output_dir + " does not exists and will be created")
    
    # Create the directory
    os.makedirs(output_dir)
    
df_flow_probs.to_csv(output_path)